# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [15]:
# import libraries
import scipy
import os
import numpy as np
import matplotlib.pyplot as plt
from sqlalchemy import create_engine
import nltk
import pandas as pd
import re

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.metrics import classification_report, accuracy_score, fbeta_score, make_scorer, recall_score, precision_score
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer, TfidfVectorizer


nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer

from scipy.stats.mstats import gmean

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\COIN1\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\COIN1\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\COIN1\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\COIN1\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [2]:
# load data from database
cwd = os.getcwd()
dbwd = cwd + '\\data\\' + 'DisasterResponse.db'

engine = create_engine('sqlite:///'+dbwd)
df = pd.read_sql_table('df', engine)

#df.head()


In [3]:
X = df['message']
Y = df.iloc[:, 4:]

### 2. Write a tokenization function to process your text data

In [4]:
def tokenize(text):
    '''
    TOKENIZE:
    @params
        text: the input text
    @return:
        clean_text: the cleaned text
    '''
    
    # normalize text
    text = re.sub(r'[^a-zA-Z0-9]', ' ',text.lower())
    
    words = word_tokenize(text)
    
    tokens = [w for w in words if w not in stopwords.words("english")]
    
    # sterm and lemmatize
    lemmatizer = WordNetLemmatizer()
    
    clean_tokens = []
    for t in tokens:
        clean_t = lemmatizer.lemmatize(t).strip()
        clean_tokens.append(clean_t)
    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [5]:
# use MultiOutputClassifier in sklearn to build up the pipeline

pipeline = Pipeline([
        ('text_pipeline', Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer())
        ])),
        ('clf', MultiOutputClassifier(AdaBoostClassifier()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.2,random_state=42)

In [7]:
# fit the model
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('text_pipeline',
                 Pipeline(memory=None,
                          steps=[('vect',
                                  CountVectorizer(analyzer='word', binary=False,
                                                  decode_error='strict',
                                                  dtype=<class 'numpy.int64'>,
                                                  encoding='utf-8',
                                                  input='content',
                                                  lowercase=True, max_df=1.0,
                                                  max_features=None, min_df=1,
                                                  ngram_range=(1, 1),
                                                  preprocessor=None,
                                                  stop_words=None,
                                                  strip_accents=None,
                                                  token_patter

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [8]:
# predict the model
y_pred = pipeline.predict(X_test)

In [9]:
categories_names = list(Y)
for i in range(35):
    print(categories_names[i], '\n', classification_report(y_test.iloc[:,i], y_pred[:, i]))

related 
               precision    recall  f1-score   support

           0       0.64      0.09      0.15      1246
           1       0.77      0.98      0.86      3948
           2       0.44      0.19      0.27        42

    accuracy                           0.76      5236
   macro avg       0.62      0.42      0.43      5236
weighted avg       0.73      0.76      0.69      5236

request 
               precision    recall  f1-score   support

           0       0.90      0.96      0.93      4291
           1       0.75      0.49      0.59       945

    accuracy                           0.88      5236
   macro avg       0.82      0.73      0.76      5236
weighted avg       0.87      0.88      0.87      5236

offer 
               precision    recall  f1-score   support

           0       1.00      1.00      1.00      5211
           1       0.00      0.00      0.00        25

    accuracy                           0.99      5236
   macro avg       0.50      0.50      0.50   

weather_related 
               precision    recall  f1-score   support

           0       0.89      0.96      0.92      3804
           1       0.86      0.67      0.75      1432

    accuracy                           0.88      5236
   macro avg       0.87      0.81      0.84      5236
weighted avg       0.88      0.88      0.88      5236

floods 
               precision    recall  f1-score   support

           0       0.96      0.99      0.98      4822
           1       0.86      0.56      0.68       414

    accuracy                           0.96      5236
   macro avg       0.91      0.77      0.83      5236
weighted avg       0.95      0.96      0.95      5236

storm 
               precision    recall  f1-score   support

           0       0.95      0.98      0.97      4733
           1       0.78      0.52      0.62       503

    accuracy                           0.94      5236
   macro avg       0.87      0.75      0.80      5236
weighted avg       0.93      0.94      

## Let's see the mdel telling us something reasonable

In [10]:
def multioutput_fscore(y_true, y_pred, beta=1):
    '''
    Calculates the f1 score and report
    '''
    score_list = []
    if isinstance(y_pred, pd.DataFrame) == True:
        y_pred = y_pred.values
    if isinstance(y_true, pd.DataFrame) == True:
        y_true = y_true.values
    
    for column in range(0, y_true.shape[1]):
        score = fbeta_score(y_true[:, column], y_pred[:, column], beta, average='weighted')
        score_list.append(score)
    
    f1score_np = np.asarray(score_list)
    f1score_np = f1score_np[f1score_np < 1]
    f1score = gmean(f1score_np)
    
    return f1score

In [11]:
multi_f1 = multioutput_fscore(y_test, y_pred, beta=1)
overall_accuracy = (y_pred == y_test).mean().mean()

print('overall accuracy: {:.2f}% \n'.format(overall_accuracy*100))
print('F1 score: {:.2f}% \n'.format(multi_f1*100))

overall accuracy: 94.48% 

F1 score: 93.30% 



### 6. Improve your model
Use grid search to find better parameters. 

In [12]:
parameters = {
         'text_pipeline__tfidf__use_idf': (True, False),
         'clf__estimator__n_estimators': [10, 50],
         'clf__estimator__learning_rate': [0.1, 1],
}
# create grid search object
cv = GridSearchCV(pipeline, param_grid=parameters, n_jobs=-1)
cv.estimator.get_params().keys()

dict_keys(['memory', 'steps', 'verbose', 'text_pipeline', 'clf', 'text_pipeline__memory', 'text_pipeline__steps', 'text_pipeline__verbose', 'text_pipeline__vect', 'text_pipeline__tfidf', 'text_pipeline__vect__analyzer', 'text_pipeline__vect__binary', 'text_pipeline__vect__decode_error', 'text_pipeline__vect__dtype', 'text_pipeline__vect__encoding', 'text_pipeline__vect__input', 'text_pipeline__vect__lowercase', 'text_pipeline__vect__max_df', 'text_pipeline__vect__max_features', 'text_pipeline__vect__min_df', 'text_pipeline__vect__ngram_range', 'text_pipeline__vect__preprocessor', 'text_pipeline__vect__stop_words', 'text_pipeline__vect__strip_accents', 'text_pipeline__vect__token_pattern', 'text_pipeline__vect__tokenizer', 'text_pipeline__vect__vocabulary', 'text_pipeline__tfidf__norm', 'text_pipeline__tfidf__smooth_idf', 'text_pipeline__tfidf__sublinear_tf', 'text_pipeline__tfidf__use_idf', 'clf__estimator__algorithm', 'clf__estimator__base_estimator', 'clf__estimator__learning_rate', 

In [13]:
cv.fit(X_train, y_train)

GridSearchCV(cv=None, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('text_pipeline',
                                        Pipeline(memory=None,
                                                 steps=[('vect',
                                                         CountVectorizer(analyzer='word',
                                                                         binary=False,
                                                                         decode_error='strict',
                                                                         dtype=<class 'numpy.int64'>,
                                                                         encoding='utf-8',
                                                                         input='content',
                                                                         lowercase=True,
                                                                         max_df=1.0,
  

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [14]:
y_pred_cv = cv.predict(X_test)

In [17]:
for i in range(35):
    print(categories_names[i], '\n', classification_report(y_test.iloc[:,i], y_pred_cv[:, i]))

related 
               precision    recall  f1-score   support

           0       0.64      0.09      0.15      1246
           1       0.77      0.98      0.86      3948
           2       0.44      0.19      0.27        42

    accuracy                           0.76      5236
   macro avg       0.62      0.42      0.43      5236
weighted avg       0.73      0.76      0.69      5236

request 
               precision    recall  f1-score   support

           0       0.90      0.96      0.93      4291
           1       0.75      0.49      0.59       945

    accuracy                           0.88      5236
   macro avg       0.82      0.73      0.76      5236
weighted avg       0.87      0.88      0.87      5236

offer 
               precision    recall  f1-score   support

           0       1.00      1.00      1.00      5211
           1       0.00      0.00      0.00        25

    accuracy                           0.99      5236
   macro avg       0.50      0.50      0.50   

weather_related 
               precision    recall  f1-score   support

           0       0.89      0.96      0.92      3804
           1       0.86      0.67      0.75      1432

    accuracy                           0.88      5236
   macro avg       0.87      0.81      0.84      5236
weighted avg       0.88      0.88      0.88      5236

floods 
               precision    recall  f1-score   support

           0       0.96      0.99      0.98      4822
           1       0.86      0.56      0.68       414

    accuracy                           0.96      5236
   macro avg       0.91      0.77      0.83      5236
weighted avg       0.95      0.96      0.95      5236

storm 
               precision    recall  f1-score   support

           0       0.95      0.98      0.97      4733
           1       0.78      0.52      0.62       503

    accuracy                           0.94      5236
   macro avg       0.87      0.75      0.80      5236
weighted avg       0.93      0.94      

In [18]:
multi_f1 = multioutput_fscore(y_test, y_pred_cv, beta=1)
overall_accuracy = (y_pred_cv == y_test).mean().mean()

print('overall accuracy: {:.2f}% \n'.format(overall_accuracy*100))
print('F1 score: {:.2f}% \n'.format(multi_f1*100))

overall accuracy: 94.48% 

F1 score: 93.30% 



### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.